In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from pandas_profiling import ProfileReport
import plotly.figure_factory as ff
import plotly.graph_objects as go
import datetime 
from dateutil.parser import parse
import nbformat
from functools import reduce
import smtplib, ssl
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [9]:
RAW_DATA_FOLDER = 'data/raw/'
OUTPUT_DATA_FOLDER = 'data/output/'
REPORT_FOLDER = 'reports/'


fund_list = pd.read_csv(os.path.join(RAW_DATA_FOLDER, 'fund_list.csv'))
emails = pd.read_csv(os.path.join(RAW_DATA_FOLDER, 'emails.csv'))

In [28]:
port = 465  # For SSL
password = input("Type your password and press enter: ")

In [10]:
fund_list = fund_list[fund_list['Known contacts/do not contact'] != 'x']

In [11]:
def clean_domain(x):
    x = x.replace('http://','').replace('https://','').replace('www.','').split("/")[0]
    return x

In [12]:
fund_list['Fund domain'] = fund_list['Fund domain'].astype('str')

In [13]:
fund_list['Fund domain'] = fund_list['Fund domain'].apply(lambda x : clean_domain(x))

In [14]:
fund_list = fund_list.drop('Known contacts/do not contact', 1)
fund_list.to_csv(os.path.join(OUTPUT_DATA_FOLDER, 'fund_list_V2.csv'), index=False)

In [15]:
emails = emails.drop('Unnamed: 0', 1)

In [19]:
exclude_array = ["contact", "office", "info", "reception", "jobs", "collaborate", "summaries", "press", "webmaster", "sales", "website", "marketing", "press", "events.breakoff", "ad"]

In [20]:
for exclude in exclude_array:
    emails = emails[~emails["0"].str.contains(exclude)]

In [25]:
emails = emails.reset_index().drop('index', 1).rename(columns={'0': 'emails'})

In [26]:
emails.to_csv(os.path.join(OUTPUT_DATA_FOLDER, 'emails_V2.csv'), index=False)

In [ ]:
sender_email = "my@gmail.com"
receiver_email = "your@gmail.com"
password = input("Type your password and press enter:")

message = MIMEMultipart("alternative")
message["Subject"] = "multipart test"
message["From"] = sender_email
message["To"] = receiver_email

# Create the plain-text and HTML version of your message
text = """\
Hi,
How are you?
Real Python has many great tutorials:
www.realpython.com"""
html = """\
<html>
  <body>
    <p>Hi,<br>
       How are you?<br>
       <a href="http://www.realpython.com">Real Python</a> 
       has many great tutorials.
    </p>
  </body>
</html>
"""

# Turn these into plain/html MIMEText objects
part1 = MIMEText(text, "plain")
part2 = MIMEText(html, "html")

# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
message.attach(part1)
message.attach(part2)

# Create secure connection with server and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(
        sender_email, receiver_email, message.as_string()
    )